In [1]:
from adam.core.computations import KinDynComputations
from adam.core.computationsWithHardwareParameters import KinDynComputationsWithHardwareParameters
from adam.geometry import utils
import numpy as np
import casadi as cs
import gym_ignition_models

In [2]:
def SX2DM(sx):
    return cs.DM(sx)

## Import the robot .urdf

In [3]:
urdf_path = gym_ignition_models.get_model_file("iCubGazeboV2_5")
# The joint list
joints_name_list = [
    'torso_pitch', 'torso_roll', 'torso_yaw', 'l_shoulder_pitch',
    'l_shoulder_roll', 'l_shoulder_yaw', 'l_elbow', 'r_shoulder_pitch',
    'r_shoulder_roll', 'r_shoulder_yaw', 'r_elbow', 'l_hip_pitch', 'l_hip_roll',
    'l_hip_yaw', 'l_knee', 'l_ankle_pitch', 'l_ankle_roll', 'r_hip_pitch',
    'r_hip_roll', 'r_hip_yaw', 'r_knee', 'r_ankle_pitch', 'r_ankle_roll'
]
# Specify the root link
root_link = 'root_link'
kinDyn = KinDynComputations(urdf_path, joints_name_list, root_link)
kinDynWithHardware = KinDynComputationsWithHardwareParameters(urdf_path, joints_name_list, root_link)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
r_hip_pitch
r_hip_roll
r_leg_ft_sensor
r_hip_yaw
r_upper_leg_back_contact_fixed_joint
r_upper_leg_dh_frame_fixed_joint
r_knee
r_ankle_pitch
r_ankle_roll
r_foot_ft_sensor
r_sole_fixed_joint
l_hip_pitch
l_hip_roll
l_leg_ft_sensor
l_hip_yaw
l_upper_leg_back_contact_fixed_joint
l_upper_leg_dh_frame_fixed_joint
l_knee
l_ankle_pitch
l_ankle_roll
l_foot_ft_sensor
l_sole_fixed_joint
torso_pitch
torso_roll
torso_yaw
r_shoulder_pitch
r_shoulder_roll
r_shoulder_yaw
r_arm_ft_sensor
r_elbow
r_wrist_prosup
r_forearm_dh_frame_fixed_joint
r_wrist_pitch
r_wrist_yaw
r_hand_dh_frame_fixed_joint
l_shoulder_pitch
l_shoulder_roll
l_shoulder_yaw
l_arm_ft_sensor
l_elbow
l_wrist_prosup
l_forearm_dh_frame_fixed_joint
l_wrist_pitch
l_wrist_yaw
l_hand_dh_frame_fixed_joint
neck_pitch
neck_roll
neck_yaw
base_fixed_joint
l_foot_dh_frame_fixed_joint
r_foot_dh_frame_fixed_joint
0
torso_pitch
1
torso_roll
2
torso_yaw
3
l_shoulder_pitch
4
l_shoulder_roll
5
l_sho

The class `KinDynComputations` contains some algorithms for computing kinematics and dynamics quantities.

In [4]:
# Set joints and base informations
xyz = (np.random.rand(3) - 0.5) * 5
rpy = (np.random.rand(3) - 0.5) * 5
H_b = utils.H_from_PosRPY(xyz, rpy)
v_b = (np.random.rand(6) - 0.5) * 5
s = (np.random.rand(len(joints_name_list)) - 0.5) * 5
s_dot = (np.random.rand(len(joints_name_list)) - 0.5) * 5
masses = (np.random.rand(kinDyn.tree.N))*0.1
InertiaMatrix = (np.random.rand(3,3))*0.2

## Mass Matrix

In [5]:
M_hardware = kinDynWithHardware.mass_matrix_fun()
M = kinDyn.mass_matrix_fun()
print('Mass matrix with hardware:\n', SX2DM(M_hardware(H_b, s, masses, InertiaMatrix)))
print('Mass matrix w\o:\n', SX2DM(M(H_b, s)))

Mass matrix with hardware:
 
[[1.92024, -2.97269e-08, -1.27282e-07, -7.53884e-10, 0.0324846, -0.118696, 0.0926189, -0.127, 0.00580627, -0.00944629, 0.00224315, -0.00670967, 0.0326201, 0.00757346, -0.0149688, 0.000615405, 0.00984619, 0.0275258, -0.0417682, 0.0252166, -0.00927356, 0.00184144, 0.00229235, 0.00426969, -0.0753962, 0.00685935, -0.0241839, 0.000941516, -0.000647513], 
 [-2.97269e-08, 1.92024, 6.70624e-08, -0.0324846, -5.68524e-09, -0.00463586, 0.0495318, 0.117231, -0.0472428, -0.0106663, 0.00532304, -0.00747305, -0.0089001, -0.0209732, 0.00947542, -0.00356346, -0.00101884, 0.0357911, 0.024783, 0.020486, -0.0296368, 0.00126634, -0.00369316, 0.0273446, -0.0173931, 0.00273547, -0.0273206, 0.00443947, -0.00207013], 
 [-1.27282e-07, 6.70624e-08, 1.92024, 0.118696, 0.00463585, 6.43912e-09, -0.177824, -0.0177602, 0.0625465, -0.0269349, -0.0326783, -0.0264303, -0.0063375, -0.00473356, -0.042124, -0.00296058, 0.00460673, -0.0216659, -0.00738632, -0.0132713, -0.0411234, 0.0042333, 0.00

## Centroidal Momentum Matrix

In [6]:
Jcm_hardware = kinDynWithHardware.centroidal_momentum_matrix_fun()
Jcmm = kinDyn.centroidal_momentum_matrix_fun()
print('Centroidal Momentum Matrix with Hardware:\n', SX2DM(Jcm_hardware(H_b, s, masses, InertiaMatrix)))
print('Centroidal Momentum Matrix w\o Hardware: \n', SX2DM(Jcmm(H_b, s)))

Centroidal Momentum Matrix with Hardware:
 
[[1.92024, -2.97269e-08, -1.27282e-07, -7.53884e-10, 0.0324846, -0.118696, 0.0926189, -0.127, 0.00580627, -0.00944629, 0.00224315, -0.00670967, 0.0326201, 0.00757346, -0.0149688, 0.000615405, 0.00984619, 0.0275258, -0.0417682, 0.0252166, -0.00927356, 0.00184144, 0.00229235, 0.00426969, -0.0753962, 0.00685935, -0.0241839, 0.000941516, -0.000647513], 
 [-2.97269e-08, 1.92024, 6.70624e-08, -0.0324846, -5.68524e-09, -0.00463586, 0.0495318, 0.117231, -0.0472428, -0.0106663, 0.00532304, -0.00747305, -0.0089001, -0.0209732, 0.00947542, -0.00356346, -0.00101884, 0.0357911, 0.024783, 0.020486, -0.0296368, 0.00126634, -0.00369316, 0.0273446, -0.0173931, 0.00273547, -0.0273206, 0.00443947, -0.00207013], 
 [-1.27282e-07, 6.70624e-08, 1.92024, 0.118696, 0.00463585, 6.43912e-09, -0.177824, -0.0177602, 0.0625465, -0.0269349, -0.0326783, -0.0264303, -0.0063375, -0.00473356, -0.042124, -0.00296058, 0.00460673, -0.0216659, -0.00738632, -0.0132713, -0.0411234, 

NameError: name 'Jcm' is not defined

## Center of mass position

In [ ]:
CoM = kinDyn.CoM_position_fun()
CoM_hardware = kinDynWithHardware.CoM_position_fun()
print('Center of Mass position:\n', SX2DM(CoM(H_b, s)))
print('Center of Mass position:\n', SX2DM(CoM_hardware(H_b, s, masses)))

root_link


KeyError: 'root_link'

## Total Mass

In [ ]:
print('Total mass:\n', kinDyn.get_total_mass())

## Jacobian

In [ ]:
J = kinDyn.jacobian_fun('l_sole')
print('Jacobian of the left sole:\n', SX2DM(J(H_b, s)))

## Relative jacobian

In [ ]:
J_r = kinDyn.relative_jacobian_fun('l_sole')
print('Jacobian between the root link and left sole:\n', SX2DM(J_r(s)))

## Forward kinematics

In [ ]:
H = kinDyn.forward_kinematics_fun('l_sole')
print('Forward kinematics to the left sole:\n', SX2DM(H(H_b, s)))

## Bias force term

In [ ]:
h = kinDyn.bias_force_fun()
print('Bias force term:\n', SX2DM(h(H_b, s, v_b, s_dot, InertiaMatrix)))

## Coriolis term

In [ ]:
C = kinDyn.coriolis_term_fun()
print('Coriolis force term:\n', SX2DM(C(H_b, s, v_b, s_dot, InertiaMatrix)))

## Gravity term

In [ ]:
G = kinDyn.gravity_term_fun()
print('Gravity term:\n', SX2DM(G(H_b, s, InertiaMatrix)))

In [ ]:
opti = cs.Opti()
Inertia_optiVar = opti.variable(3,3, 'symmetric')
masses_optVar = opti.variable(kinDyn.tree.N)
q_param = opti.parameter(kinDyn.NDoF)
H_b_param = opti.parameter(4,4)
H_b_new = np.random.rand(4,4)
H_b_new[:3, :3] = utils.R_from_RPY(rpy)
H_b_new[:3, 3] = xyz
M_f = kinDyn.mass_matrix_fun()
a = M_f(H_b_param,q_param, masses_optVar, Inertia_optiVar)
opti.minimize((a[1,1]))
opti.set_value(H_b_param, H_b_new)
opti.set_value(q_param, s)
opti.solver('ipopt')
sol = opti.solve()

print(sol.Inertia_optiVar)